# SciPy

The SciPy library is one of the core packages that make up the SciPy stack. t provides many user-friendly and efficient numerical routines such as routines for numerical integration and optimization.

Library document: [http://www.scipy.org/scipylib/index.html](http://www.scipy.org/scipylib/index.html)

In [1]:
# If we need to display the graphs on this page, use following code
%matplotlib inline
from pylab import *

In [2]:
import numpy as np
from scipy.integrate import quad, dblquad, tplquad

## 1. Interation

This shows how to calcuate integration of a function with SciPy.

Integral: [https://en.wikipedia.org/wiki/Integral](https://en.wikipedia.org/wiki/Integral)

In [3]:
# Calculate integration of a function, return the integrate and absolute error
val, abserr = quad(lambda x: exp(-x ** 2), Inf, Inf)
val, abserr

(0.0, 0.0)

## 1. Spacial functions

This module provide special functions. Here is the list of special functions: [https://docs.scipy.org/doc/scipy/reference/special.html#module-scipy.special](https://docs.scipy.org/doc/scipy/reference/special.html#module-scipy.special)

Frequently used ones are:

- Baseel function: `scipy.special.jn()`
- Elliptic function: `scipy.special.ellipj()`
- Gamma function: `scipy.special.gamma()`
- Erf, the area under a Gaussian curve: `scipy.special.erf()`

## 2. Linear algebra operations

The `scipy.linalg` module provides standard linear algebra operations, relying on an underlying efficient implementation (BLAS, LAPACK).

- Compute determinant of a square matrix

In [4]:
from scipy import  linalg
arr = np.array([[1,2],
                [3,4]])

In [5]:
linalg.det(arr)

-2.0

In [6]:
arr2 = np.array([[3,2], 
                 [6,4]])
linalg.det(arr2)

6.661338147750939e-16

In [7]:
# The matrix should be a square matrix
linalg.det(np.ones((3,4)))

ValueError: expected square matrix

- Compute the inverse of a sqare matrix

In [ ]:
arr = np.array([[1,2],
                [3,4]])

In [ ]:
iarr = linalg.inv(arr)
iarr

In [ ]:
# product of a matrix and it's inverse is an eye matrix
np.allclose(np.dot(arr, iarr), np.eye(2))

In [ ]:
# Cannot computing the inverse of a singular matrix (its determinant is zero)
linalg.inv(arr2)

- More advance operations

SVD is commonly used in statistics and signal processing. Many other standard decompositions (QR, LU, Cholesky, Schur), as well as solvers for linear systems, are available in [`scipy.linalg`](http://docs.scipy.org/doc/scipy/reference/linalg.html#module-scipy.linalg).

In [ ]:
# singular-value decomposition (SVD)
arr = np.arange(9).reshape((3,3)) + np.diag([1, 0, 1])
uarr, spec, vharr = linalg.svd(arr)

In [ ]:
# The resulting array spectrum is:
spec

In [ ]:
# The original matrix can be re-composed
sarr = np.diag(spec)
svd_mat = uarr.dot(sarr).dot(vharr)
np.allclose(svd_mat, arr)

## 3. Fast Fourier Transforms

The [`scipy.fftpack`](http://docs.scipy.org/doc/scipy/reference/fftpack.html#module-scipy.fftpack) module allows to compute fast Fourier transforms. As an illustration, a (noisy) input signal may look like:

In [ ]:
time_step = 0.02
period = 5.
time_vec = np.arange(0, 20, time_step)
sig = np.sin(2 * np.pi / period * time_vec) + 0.5 * np.random.randn(time_vec.size)

The observer doesn’t know the signal frequency, only the sampling time step of the signal sig. The signal is supposed to come from a real function so the Fourier transform will be symmetric. The `scipy.fftpack.fftfreq()` function will generate the sampling frequencies and `scipy.fftpack.fft()` will compute the fast Fourier transform:

In [ ]:
from scipy import fftpack

In [ ]:
sample_freq = fftpack.fftfreq(sig.size, d=time_step)
sig_fft = fftpack.fft(sig)

Because the resulting power is symmetric, only the positive part of the spectrum needs to be used for finding the frequency:

In [ ]:
pidxs = np.where(sample_freq > 0)
freqs = sample_freq[pidxs]
power = np.abs(sig_fft)[pidxs]

In [ ]:
import pylab as plt
plt.figure()
plt.plot(freqs, power)
plt.xlabel('Frequency [Hz]')
plt.ylabel('plower')
axes = pl.axes([0.3, 0.3, 0.5, 0.5])
plt.title('Peak frequency')
plt.plot(freqs[:8], power[:8])
plt.setp(axes, yticks=[])

The signal frequency can be found by:

In [ ]:
freq = freqs[power.argmax()]
np.allclose(freq, 1./period) # check that correct freq is found

Now the high-frequency noise will be removed from the Fourier transformed signal:

In [ ]:
sig_fft[np.abs(sample_freq) > freq] = 0

The resulting filtered signal can be computed by the `scipy.fftpack.ifft()` function:

In [ ]:
main_sig = fftpack.ifft(sig_fft)

The result can be viewed with:

In [ ]:
plt.figure()
plt.plot(time_vec, sig)
plt.plot(time_vec, main_sig, 'r', linewidth=3)
plt.xlabel('Time [s]')
plt.ylabel('Amplitude')

## 4. Optimization and fit

Optimization is the problem of finding a numerical solution to a minimization or equality. The `scipy.optimize` module provides useful algorithms for function minimization (scalar or multi-dimensional), curve fitting and root finding.

In [ ]:
from scipy import optimize

### - Finding minumum of a scalar function

In [ ]:
def f(x):
    return x ** 2 + 10*np.sin(x)

In [ ]:
# Plot the function
x = np.arange(-10, 10, 0.1)
plt.plot(x, f(x))
plt.show()

This function has a global minimum around -1.3 and a local minimum around 3.8.

The general and efficient way to find a minimum for this function is to conduct a gradient descent starting from a given initial point. The BFGS algorithm is a good way of doing this:

In [ ]:
optimize.fmin_bfgs(f, 0)

A possible issue with this approach is that, if the function has local minima the algorithm may find these local minima instead of the global minimum depending on the initial point:

In [ ]:
optimize.fmin_bfgs(f, 3, disp=0)

If we don’t know the neighborhood of the global minimum to choose the initial point, we need to resort to costlier global optimization. To find the global minimum, we use `scipy.optimize.basinhopping()` (which combines a local optimizer with stochastic sampling of starting points for the local optimizer):

In [ ]:
optimize.basinhopping(f, 0)

In [ ]:
grid = (-10, 10, 0.1)
xmin_global = optimize.brute(f, (grid,))
xmin_local = optimize.fminbound(f, 0, 10)

### - Finding the roots of a scalar function

To find a root, i.e. a point where $f(x) = 0$, of the function $f$ above we can use for example `scipy.optimize.fsolve()`:

In [ ]:
root = optimize.fsolve(f, 1)
root

Note that only one root is found. Inspecting the plot of $f$ reveals that there is a second root around -2.5. We find the exact value of it by adjusting our initial guess:

In [ ]:
root2 = optimize.fsolve(f, -2.5)
root2

### - Curve fitting

Suppose we have data sampled from $f$ with some noise:

In [ ]:
xdata = np.linspace(-10, 10, num=20)
ydata = f(xdata) + np.random.randn(xdata.size)

Now if we know the functional form of the function from which the samples were drawn ($x^2 + \sin(x)$ in this case) but not the amplitudes of the terms, we can find those by least squares curve fitting. First we have to define the function to fit:

In [ ]:
def f2(x, a, b):
    return a*x**2 + b*np.sin(x)

Then we can use `scipy.optimize.curve_fit()` to find a and b:

In [ ]:
guess = [2,2]
params, params_covariance = optimize.curve_fit(f2, xdata, ydata, guess)
params

Now we have found the minima and roots of f and used curve fitting on it, we put all those resuls together in a single plot:

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(x, f(x), 'b-', label="f(x)")
ax.plot(x, f2(x, *params), 'r--', label="Curve fit result")
xmins = np.array([xmin_global[0], xmin_local])
ax.plot(xmins, f(xmins), 'go', label="Minima")
roots = np.array([root, root2])
ax.plot(roots, f(roots), 'kv', label="Roots")
ax.legend()
ax.set_xlabel('x')
ax.set_ylabel('f(x)')

## 5. Statistics and random numbers
The module `scipy.stats` contains statistical tools and probabilistic descriptions of random processes. 

### Histogram and probability density function